In [ ]:
# Auto-reload frequently changed files
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact
from os.path import join

from constants import COLUMNS
from utils import (
    read_latest_daily_counts_df, preprocess_daily_counts_df_for_vis,
    read_latest_demographics_df, preprocess_demographics_df_for_vis, 
    read_latest_diagnoses_df,
    read_latest_labs_df, preprocess_labs_df_for_vis,
    read_loinc_df, read_icd_df,
    apply_theme
)

# Dataset

### Demographics

In [ ]:
dem_df = read_latest_demographics_df()
dem_df = preprocess_demographics_df_for_vis(dem_df)

DEM_SITE_IDS = dem_df[COLUMNS.SITE_ID].unique().tolist()

# Columns to use
siteid = COLUMNS.SITE_ID
sex = COLUMNS.SEX
total_patients = COLUMNS.TOTAL_PATIENTS
age_group = COLUMNS.AGE_GROUP
num_patients = COLUMNS.NUM_PATIENTS

dem_df.head()

### DailyCounts

In [ ]:
dai_df = read_latest_daily_counts_df()
dai_df = preprocess_daily_counts_df_for_vis(dai_df)

DAI_SITE_IDS = dai_df[COLUMNS.SITE_ID].unique().tolist()

# Columns to use
siteid = COLUMNS.SITE_ID
date = COLUMNS.DATE
new_positive_cases = "New positive cases"
patients_in_icu = "Patients in ICU"
new_deaths = "New deaths"
category = "category"
num_patients = COLUMNS.NUM_PATIENTS

dai_df

### Lab

In [ ]:
lab_df = read_latest_labs_df()
lab_df = preprocess_labs_df_for_vis(lab_df)

LAB_SITE_IDS = lab_df[COLUMNS.SITE_ID].unique().tolist()

# Columns to use

lab_df

### Diagnoses

In [ ]:
dia_df = read_latest_diagnoses_df()
SITE_IDS = dia_df[COLUMNS.SITE_ID].unique().tolist()

# Columns
siteid = COLUMNS.SITE_ID
icd_code = COLUMNS.ICD_CODE
icd_version = COLUMNS.ICD_VERSION
num_patients = COLUMNS.NUM_PATIENTS

dia_df[COLUMNS.ICD_CODE] = dia_df[COLUMNS.ICD_CODE].apply(lambda x: x.replace(".", ""))

icd_df = read_icd_df()
dia_df = dia_df.merge(icd_df, how="left", left_on='icd_code', right_on='ICDcode')

# Handle the missing data
dia_df.loc[pd.isna(dia_df["ICDdescription"]), "ICDdescription"] = dia_df.loc[pd.isna(dia_df["ICDdescription"]), "icd_code"]
dia_df.loc[pd.isna(dia_df["Category"]), "Category"] = dia_df.loc[pd.isna(dia_df["Category"]), "icd_code"]

# Consistent capitalization
dia_df["ICDdescription"] = dia_df["ICDdescription"].apply(lambda x: x.capitalize())
dia_df["Category"] = dia_df["Category"].apply(lambda x: x.capitalize())

dia_df